In [ ]:
from genetic_algorithm import *
import re
import json
import numpy as np
from numpy.random import default_rng
from collections import Counter
import copy
import matplotlib.pyplot as plt
from itertools import groupby

## hexadecimal generator

In [ ]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 100

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [ ]:
## world size
world_size_x = 128
world_size_y = 128

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)


In [ ]:
individuals_sum_dup = sum_duplicated_neurons(individuals)

## initial brain and position generator

In [ ]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## brain generator
for individual in individuals_sum_dup:
    
    ## init brain neuron generator
    print(individual)
    ## preprocessing
    edges = individuals_sum_dup[individual]
    edges = [tuple(edges[i]) for i in edges]
    edges_no_weight = [i[:-1] for i in edges]
    
    G = nx.MultiDiGraph()
    G.add_edges_from(edges)
    
    out_list = list(set([i[1] for i in edges if 'out' in i[1]]))
    init_list = list(set([i[0] for i in edges if 'in' in i[0]]))

    # generate dict of paths
    dic_of_paths = generate_dict_of_paths(out_list, init_list, G)

    lis = filtered_neurons_paths(dic_of_paths)
    
    dic = append_weight_to_neurons_in_path(lis, edges_no_weight, edges)

    initial_dic_of_output_neurons[individual] = dic
        
        

#     print(dic)
    print()

mid predecessors variants

In [28]:
l = [{'mid1': {'in2':1}},
    {'mid1': {'mid1':1}},
    {'mid1': {'mid2':1}},
    {'mid1': {'in1':1, 'in2':2}},
    {'mid1': {'in1':1, 'mid1':2}},
    {'mid1': {'in1':1, 'mid2':2}},
    {'mid1': {'mid1':1, 'mid2':2}},
    {'mid1': {'mid2':1, 'mid3':2}},
    {'mid1': {'in1':1, 'in2':2, 'mid1':3}},
    {'mid1': {'in1':1, 'in2':2, 'mid2':3}},
    {'mid1': {'in1':1, 'mid1':2, 'mid2':3}},
    {'mid1': {'in1':1, 'mid2':2, 'mid3':3}},
    {'mid1': {'mid1':1, 'mid2':2, 'mid3':3}},
    {'mid1': {'in1':1, 'in2':2, 'mid1':3, 'mid2':4}},
    {'mid1': {'in1':1, 'in2':2, 'mid2':3, 'mid3':4}},
    {'mid1': {'in1':1, 'mid1':2, 'mid2':3, 'mid3':4}},
    {'mid1': {'in1':1, 'in2':2, 'mid1':3, 'mid2':4, 'mid3':5}}]

In [29]:
key = 'mid1'
sample_list = ['in1', 'in2', 'mid1', 'mid2', 'mid3']

in_plus_loop = []
mid = []

for i in sample_list:
    if 'in' in i or key in i:
        in_plus_loop.append(i)
    elif 'mid' in i and key not in i:
        mid.append(i) 
        
in_plus_loop, mid

(['in1', 'in2', 'mid1'], ['mid2', 'mid3'])

In [30]:
for dic in l:
    for key in dic:
        print(dic[key])
        print('sum weights', set(dic[key]).difference(set(mid)), np.tanh(sum(dic[key].values())))
        
        print('replace with weight', set(dic[key]).difference(set(in_plus_loop)))
        print()

{'in2': 1}
sum weights {'in2'} 0.7615941559557649
replace with weight set()

{'mid1': 1}
sum weights {'mid1'} 0.7615941559557649
replace with weight set()

{'mid2': 1}
sum weights set() 0.7615941559557649
replace with weight {'mid2'}

{'in1': 1, 'in2': 2}
sum weights {'in1', 'in2'} 0.9950547536867305
replace with weight set()

{'in1': 1, 'mid1': 2}
sum weights {'in1', 'mid1'} 0.9950547536867305
replace with weight set()

{'in1': 1, 'mid2': 2}
sum weights {'in1'} 0.9950547536867305
replace with weight {'mid2'}

{'mid1': 1, 'mid2': 2}
sum weights {'mid1'} 0.9950547536867305
replace with weight {'mid2'}

{'mid2': 1, 'mid3': 2}
sum weights set() 0.9950547536867305
replace with weight {'mid2', 'mid3'}

{'in1': 1, 'in2': 2, 'mid1': 3}
sum weights {'in1', 'in2', 'mid1'} 0.9999877116507956
replace with weight set()

{'in1': 1, 'in2': 2, 'mid2': 3}
sum weights {'in1', 'in2'} 0.9999877116507956
replace with weight {'mid2'}

{'in1': 1, 'mid1': 2, 'mid2': 3}
sum weights {'in1', 'mid1'} 0.999987711

In [ ]:
dic = {'mid0': {'mid0': 1.8478254234701663, 'in1': 1.7749427743018464, 'mid2': -3.906241416145277}, 'mid1': -0.9999224478418683, 'mid2': {'in1': -0.07153975278498398, 'mid1': -0.792797192125744}, 'out1': {'in1': -3.3650541736609187, 'mid0': 0.92733099343812, 'mid2': -3.8355562337860523}}
dic

In [ ]:
dic = {'mid0': {'mid0': 1.8478254234701663, 'in1': 1.7749427743018464, 'mid2': -3.906241416145277}, 'mid1': -0.9999224478418683, 'mid2': {'in1': -0.07153975278498398, 'mid1': -0.792797192125744}, 'out1': {'in1': -3.3650541736609187, 'mid0': 0.92733099343812, 'mid2': -3.8355562337860523}}

mid_value = {}

for key in dic:
#     print(key)
    if 'mid' in key and isinstance(dic[key], float):
        mid_value[key] = dic[key]

    elif 'mid' in key and isinstance(dic[key], dict):
        for sub_key in dic[key]:
            if sub_key in mid_value:
                dic[key][sub_key] += mid_value[sub_key]

for key in mid_value:
    del dic[key]

# dic_list = {}
# for key in dic:
#     lista = []
#     for sub_key in dic[key]:
#         if sub_key in dic:
#             lista.append(sub_key)
#     dic_list[key] = lista
    
# def sum_weights(dic):
#     for key in dic_list:
#         if dic_list:
#             sum_weights

            
    

dic

In [ ]:
any(dic['mid0']) in dic

In [ ]:
 np.tanh(sum([-0.9999224478418683, 1.8478254234701663, -0.9530708314545707, 1.7749427743018464]))

In [ ]:
out_to_weight(dic)

In [ ]:
len({'in1': -0.07153975278498398, 'mid1': -0.792797192125744})

In [ ]:
dic

In [ ]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
for individual in individuals_sum_dup_no_self_loop:
    
    ## init brain neuron generator
    
    ## preprocessing
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    remove_mid_with_no_predecessor(edges) 
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
    mid_to_weight(dic)
    out_to_weight(dic)
       
    dic = {k:v for k, v in dic.items() if 'out' in k}    
    
    initial_dic_of_output_neurons[individual] = {}
    initial_dic_of_output_neurons[individual]['output'] = dic
    
    ## position
#     initial_dic_of_output_neurons[individual]['position'] = {}
#     initial_dic_of_output_neurons[individual]['position']['x'] = x[individual]
#     initial_dic_of_output_neurons[individual]['position']['y'] = y[individual] 
    
    ## position history
    initial_dic_of_output_neurons[individual]['history_position'] = [(x[individual], y[individual])]

In [ ]:
initial_dic_of_output_neurons

In [ ]:
a = [(3, 0),(4,1),(5,2)]

In [ ]:
def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

print(slope_intercept(3,0,2,1))

In [ ]:
a

In [ ]:
if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
print(x)

In [ ]:
a[-2][0]